In [1]:
import helper
import matplotlib.pyplot as plt
from keras.applications import *
from keras.callbacks import EarlyStopping
import os

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
#设置各种参数
train_path = ['./data/train2/cat', './data/train2/dog']
test_path ='./data/test1/test1'
img_size =(299,299)
layer_num = 249
model_image ='./models/model_image_fine_tuning_inceptionv3_0403.png'
model_weights_file = './models/weights_fine_tuning_inceptionv3_no_outliers_0403.h5'
template_csv_path = './predicts/sample_submission.csv'
target_csv_path = './predicts/pred_fine_tuning_inceptionv3_no_outliers_0403.csv'
MODEL = inception_v3.InceptionV3
preprocess_func = inception_v3.preprocess_input

In [3]:
#获取训练集数据
X_train, y_train, image_files= helper.get_train_input_from_folder_with_subclasses(train_path, img_size, lambda_func=preprocess_func)
print("finished")

(24959, 299, 299, 3)
(24959, 1)
finished
finished


In [4]:
#构造模型，锁定base_model所有层
model = helper.get_fine_tuning_first_model(MODEL)

#可视化模型
helper.visualize_model(model, model_image)
print("finished")

start
(?, ?, ?, 3)
(?, ?, ?, 2048)
finished


In [5]:
print("start")
#第一次训练新添加层权重
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(X_train, y_train, batch_size=128, epochs=8, validation_split=0.2, callbacks=[early_stopping])
print("finished")

start
Train on 19967 samples, validate on 4992 samples
Epoch 1/8
19967/19967 [==============================] - 75s 4ms/step - loss: 0.2245 - acc: 0.9256 - val_loss: 0.0787 - val_acc: 0.9792
Epoch 2/8
19967/19967 [==============================] - 68s 3ms/step - loss: 0.1011 - acc: 0.9657 - val_loss: 0.0800 - val_acc: 0.9722
Epoch 3/8
19967/19967 [==============================] - 68s 3ms/step - loss: 0.0855 - acc: 0.9692 - val_loss: 0.0738 - val_acc: 0.9742
Epoch 4/8
19967/19967 [==============================] - 68s 3ms/step - loss: 0.0790 - acc: 0.9716 - val_loss: 0.0642 - val_acc: 0.9786
Epoch 5/8
19967/19967 [==============================] - 68s 3ms/step - loss: 0.0793 - acc: 0.9708 - val_loss: 0.0840 - val_acc: 0.9702
Epoch 6/8
19967/19967 [==============================] - 68s 3ms/step - loss: 0.0738 - acc: 0.9725 - val_loss: 0.0773 - val_acc: 0.9728
Epoch 7/8
19967/19967 [==============================] - 68s 3ms/step - loss: 0.0720 - acc: 0.9728 - val_loss: 0.0823 - val_acc: 

In [6]:
print("start")
#放开若干层权重，再次训练
model = helper.get_fine_tuning_second_model(model, layer_num)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
model.fit(X_train, y_train, batch_size=128, epochs=60, validation_split=0.2, callbacks=[early_stopping])
print("finished")

start
Train on 19967 samples, validate on 4992 samples
Epoch 1/60
19967/19967 [==============================] - 76s 4ms/step - loss: 0.0682 - acc: 0.9749 - val_loss: 0.0342 - val_acc: 0.9906
Epoch 2/60
19967/19967 [==============================] - 73s 4ms/step - loss: 0.0585 - acc: 0.9784 - val_loss: 0.0323 - val_acc: 0.9908
Epoch 3/60
19967/19967 [==============================] - 73s 4ms/step - loss: 0.0540 - acc: 0.9803 - val_loss: 0.0311 - val_acc: 0.9908
Epoch 4/60
19967/19967 [==============================] - 73s 4ms/step - loss: 0.0462 - acc: 0.9832 - val_loss: 0.0282 - val_acc: 0.9920
Epoch 7/60
19967/19967 [==============================] - 73s 4ms/step - loss: 0.0435 - acc: 0.9853 - val_loss: 0.0274 - val_acc: 0.9920
Epoch 8/60
19967/19967 [==============================] - 73s 4ms/step - loss: 0.0438 - acc: 0.9837 - val_loss: 0.0271 - val_acc: 0.9920
Epoch 9/60
19967/19967 [==============================] - 73s 4ms/step - loss: 0.0423 - acc: 0.9843 - val_loss: 0.0268 - va

In [7]:
#保存模型参数
model.save_weights(model_weights_file)
del X_train
del y_train
print("finished")

finished


In [8]:
print("start")
#获取测试数据和对应文件
X_test, test_files = helper.get_input_from_folder_with_image_files(test_path, img_size, lambda_func=preprocess_func)

#获取文件basename
image_file_names = [os.path.basename(path) for path in test_files]

#预测并保存预测结果到csv
helper.predict_and_update_to_csv(model, X_test, image_file_names, template_csv_path, target_csv_path)

print("finished")

start
finished
12500/12500 [==============================] - 41s 3ms/step
finished
finished
